# Alzheimers Multi Class Classification

## Import Packages

In [ ]:
import numpy as np
import scipy as sp
import pandas as pd
import seaborn as sns
import tensorflow as tf
import keras_cv
from tensorflow.data import AUTOTUNE
import tensorflow.keras as keras
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE
from tensorflow.keras.layers import (
    RandomBrightness, RandomZoom, RandomFlip,
    Input, Conv2D, BatchNormalization, MaxPool2D, Dropout, Flatten, Dense
)

## Load Data
This data was processed in the first part of this here: https://www.kaggle.com/code/notdroid/alzheimers-dataset-processing-outliers-imbalance/notebook

In [ ]:
class_names = ['MildDemented', 'ModerateDemented', 'NonDemented', 'VeryMildDemented']
image_size = (176,208)

train_data = tf.keras.utils.image_dataset_from_directory(
    '/kaggle/input/alzheimers-dataset/training', 
    color_mode = 'grayscale',
    class_names = class_names,
    image_size = image_size,
    label_mode = 'categorical',
    batch_size = None
)

val_data = tf.keras.utils.image_dataset_from_directory(
    '/kaggle/input/alzheimers-dataset/validation', 
    color_mode = 'grayscale',
    class_names = class_names,
    image_size = image_size,
    label_mode = 'categorical',
    batch_size = None
)

## Add Data Augmentation

We have to be careful with our augmentation because each picture is taken in a similiar meanner, so we will do our augmentation in a minimal fashion. This makes it so that we still get the regularizing effect of sligthly varying the data in meaningful ways but not disrupting the models ability to analyze the way images were positioned.

In [ ]:
rescaling = keras.layers.Rescaling(1./255)

aug = keras.Sequential([
    Input(shape = (*image_size,1)),
    keras_cv.layers.AutoContrast((0,1)),
    RandomBrightness(0.15, value_range = (0,1)),
    keras_cv.layers.RandomShear(x_factor = 0.01),
    RandomZoom(0.01, fill_mode = 'constant'),
    RandomFlip('horizontal')
])

# Create Model
Here I use a class to make tuning the model easier. The data is also converted into a tensorflow dataset with best practices for optimal results and performance used.

In [ ]:
AUC = keras.metrics.AUC(curve = 'PR')

class disease_model:
    def __init__(self, model = None, weight_decay = None):
        self.model = model
        self.weight_decay = weight_decay
    
    def create_dataset(self,ds,epochs,batch_size, train = True):
        ds = ds.map(lambda image, label: (rescaling(image),label), num_parallel_calls = AUTOTUNE)
        ds = ds.cache()
        ds = ds.shuffle(100*batch_size)
        ds = ds.batch(batch_size)
        if train:
            ds = ds.map(lambda image,label: (aug(image), label), num_parallel_calls = AUTOTUNE)
        ds = ds.repeat(epochs)
        ds = ds.prefetch(AUTOTUNE)

        return ds
    def conv_block(self,filters, dropout, batchnorm = True):
        block = [
            Conv2D(filters,3, activation = 'leaky_relu', kernel_initializer = 'he_uniform', padding = 'same'),
            Conv2D(filters,3, activation = 'leaky_relu', kernel_initializer = 'he_uniform', padding = 'same')
        ]
        if batchnorm:
            block.append(BatchNormalization())
        block.append(MaxPool2D())
        block.append(Dropout(dropout))
        
        return block

    def dense_block(self,units, dropout):

        return [
            Dense(units, activation = 'leaky_relu', kernel_initializer = 'he_uniform'),
            BatchNormalization(),
            Dropout(dropout)
        ]

    def create_model(self, learning_rate = 1e-3):

        self.model = keras.Sequential([
            Input(shape = (*image_size,1)),
            *self.conv_block(16, 0.5, False),
            *self.conv_block(32, 0.2),
            *self.conv_block(64, 0.2),
            *self.conv_block(128, 0.2),
            *self.conv_block(256,0.1),
            Flatten(),
            *self.dense_block(512, 0.5),
            *self.dense_block(256, 0.35),
            *self.dense_block(128, 0.2),
            *self.dense_block(64, 0.2),
            Dense(4)
        ])
        
        opt = keras.optimizers.Adam(learning_rate, weight_decay = self.weight_decay)

        self.model.compile(loss = keras.losses.CategoricalCrossentropy(from_logits = True), optimizer = 'adam', metrics = ['accuracy',AUC])

        return self.model
    
    def train_model(self,epochs = 1, batch_size = 32, decay_step = 100, learning_rate = 1e-3):
        if not self.model:
            self.model = self.create_model(learning_rate)
        checkpointer = tf.keras.callbacks.ModelCheckpoint(
            filepath='/kaggle/working/checkpoint',
            monitor='val_auc',
            mode='max',
            save_best_only=True)
        step_decay = keras.callbacks.LearningRateScheduler(
            lambda epoch: learning_rate*10**(-int(epoch/decay_step))
        )
        train_ds = self.create_dataset(train_data,epochs,batch_size)
        val_ds = self.create_dataset(val_data,epochs,batch_size, train = False)
        history = self.model.fit(train_ds, steps_per_epoch = 10312//batch_size,
                  validation_data = val_ds, validation_steps = 1279//batch_size,
                  epochs = epochs, callbacks = [step_decay,checkpointer])

        return self.model, history

## View Model

In [ ]:
!pip install visualkeras
from visualkeras import layered_view

test_model = disease_model().create_model()

test_model.summary()
layered_view(test_model, legend=True)

# Train Model

In [ ]:
modelling_class = disease_model(weight_decay = 1e-2)
model, history = modelling_class.train_model(epochs = 150, decay_step = 50)

# Save Model

In [ ]:
checkpoint_path = '/kaggle/working/checkpoint'
best_model = keras.models.load_model(checkpoint_path)
best_model.summary()

In [ ]:
x_input = Input(shape = (*image_size,1))
x = best_model(x_input)
x_output = keras.layers.Activation('softmax')(x)

final_model = keras.Model(x_input,x_output)
final_model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy',AUC])

In [ ]:
final_model.save('AlzheimersModel2.h5')

# Evaluate Model

In [ ]:
#10312, 1279
X_val = np.zeros((1279,*image_size,1))
y_val = np.zeros((1279,4))
for i, (x, y) in enumerate(val_data):
    X_val[i] = x/255.
    y_val[i] = y

In [ ]:
def evaluate_model(model):
    y_sparse = np.argmax(y_val, axis = -1)
    recalls = []
    precisions = []
    F1_scores = []
    pred_prob = final_model.predict(X_val)
    pred_labels = np.argmax(pred_prob, axis = -1)
    
    for i, class_name in enumerate(class_names):
        T_p = np.sum((pred_labels == i)*(y_sparse == i))
        F_p = np.sum((pred_labels == i)*(y_sparse != i))
        F_n = np.sum((pred_labels !=i )*(y_sparse == i))
        
        recall = T_p/(T_p + F_n)
        precision = T_p/(T_p + F_p)
        F1 = 2/((1/recall)+(1/precision))
        
        recalls.append(recall)
        precisions.append(precisions)
        F1_scores.append(F1)
        
        print(class_name, ' recall: ', recall)
        print(class_name, ' precision: ', precision)
        print(class_name, ' F1: ', F1)
        
    macro_F1 = np.mean(F1_scores)
    
    print('Average F1: ', macro_F1)
        
evaluate_model(best_model)

Looks like we can almost always be certain of our model's results, if we try evaluating on the original validation dataset we wont get the same results because of the brightness ourliers, however if we process the dataset using the same lower fence for outliers we can achieve good results.